In [1]:
import sys
sys.path.append('../../data/automation_scripts_data/')
from automation_cv_data_processing import run_processing_data_cv
from automation_job_data_processing import run_processing_data_job
from automation_scripts_model.automation_cv_modelling import run_automation_modelling_cv
from automation_scripts_model.automation_job_modelling import run_automation_modelling_job
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\jarod\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trai

In [2]:
# Cargamos txt de cv
raw_cvs_SoftwareDev = open('../../data/raw_data/Data_CV_SoftwareDeveloper.txt',encoding='utf-8').read()
raw_cvs_NetworkEng = open('../../data/raw_data/Data_CV_NetworkEngineer.txt',encoding='utf-8').read()
raw_cvs_IADev= open('../../data/raw_data/Data_CV_IADeveloper.txt',encoding='utf-8').read()

In [3]:
# Cargamos txt de jobs
raw_jobs_SoftwareDev = open('../../data/raw_data/Data_Jobs_SoftwareDeveloper.txt',encoding='utf-8').read()
raw_jobs_NetworkEng = open('../../data/raw_data/Data_Jobs_NetworkEngineer.txt',encoding='utf-8').read()
raw_jobs_IADev= open('../../data/raw_data/Data_Jobs_IADeveloper.txt',encoding='utf-8').read()

In [4]:
# Llamamos al script de procesamiento de datos
df_cv_processed, df_cv_original = run_processing_data_cv(raw_cvs_SoftwareDev,raw_cvs_NetworkEng,raw_cvs_IADev)
df_jobs_processed, df_job_original = run_processing_data_job(raw_jobs_SoftwareDev,raw_jobs_NetworkEng,raw_jobs_IADev)

--INICIANDO PROCESAMIENTO DE DATOS CVS--
Analizando CVs y extraccion de categorias...
Fusionando categorias procesadas...
Eliminando valores innecesarios...
Estableciendo id de posicion...
Convirtiendo a Dataframe...
Limpieza inicial de datos...
Limpieza semantica de datos...
Aplicando mejoras...
--FIN DEL PROCESAMIENTO--
--INICIANDO PROCESAMIENTO DE DATOS JOBS--
Analizando Jobs y extraccion de categorias...
Fusionando categorias procesadas...
Eliminando valores innecesarios...
Estableciendo id de posicion...
Convirtiendo a Dataframe...
Limpieza inicial de datos...
Limpieza semantica de datos...
Aplicando mejoras...
--FIN DEL PROCESAMIENTO--


In [6]:
df_cv_processed.head()

,id,cv_tokens
0,0,desarrollador full stack experimentado nombre ...
1,1,desarrollador móvil junior nombre ana garcía r...
2,2,desarrollador back end especializado nombre ma...
3,3,desarrollador web front end creativo nombre da...
4,4,desarrollador juegos independiente nombre pedr...


In [6]:
df_jobs_processed.head()

,id,job_tokens
0,0,desarrollador software junior aplicaciones móv...
1,1,ingeniero software full stack tecnologías web ...
2,2,desarrollador software frontend experiencia us...
3,3,desarrollador software backend arquitectura mi...
4,4,ingeniero software juegos desarrollo videojueg...


In [11]:
# Carga modelo y generar embeddings
embeddings_cv, df_cvs = run_automation_modelling_cv(df_cv_processed)
embeddings_job, df_jobs = run_automation_modelling_job(df_jobs_processed)

**Calculo similitud del coseno**

In [12]:
def calculate_cosine(mean_embedding_1,mean_embedding_2):
    similarity_score = cosine_similarity([mean_embedding_1], [mean_embedding_2])[0, 0]
    return similarity_score

In [13]:
def clean_df_original(df_cv, df_job):
    df_cv.iloc[:, 1] = df_cv.iloc[:, 1].apply(lambda x: x.replace('\n', ' '))
    df_job.iloc[:, 1] = df_job.iloc[:, 1].apply(lambda x: x.replace('\n', ' '))
    return df_cv, df_job

In [16]:
def similitud_cv(id_cv, id_job):
    
    # Calcular la media de los embeddings
    mean_embedding_1 = np.mean(embeddings_cv[id_cv], axis=0)
    mean_embedding_2 = np.mean(embeddings_job[id_job], axis=0)

    # Calcular la similitud coseno entre los embeddings
    similarity_score = calculate_cosine(mean_embedding_1,mean_embedding_2)

    # Limpiamos los saltos de lineas '\n' de los dataframes originales (sin procesar previamente)
    df_cvs, df_jobs = clean_df_original(df_cv_original, df_job_original)
    
    # Mostrar en pantalla resultados
    print(f"La similaridad entre el CV {id_cv} y la oferta de trabajo {id_job} es de:", similarity_score)
    print('\n------------')
    print(f'CV con id {id_cv}')
    print(df_cvs['cv'][id_cv])
    print('\n------------')
    print(f'Oferta con id {id_job}')
    print(df_jobs['job'][id_job])

In [17]:
similitud_cv(8, 2)

La similaridad entre el CV 8 y la oferta de trabajo 2 es de: 0.6756893

------------
CV con id 8
Desarrollador Web Senior Nombre: Andrés Martínez  Resumen Profesional: Desarrollador web senior con más de 10 años de experiencia en el diseño y desarrollo de sitios web dinámicos y aplicaciones web. Expertise en HTML5, CSS3, JavaScript y frameworks como React.js y Vue.js. Demostrada capacidad para liderar equipos y gestionar proyectos de principio a fin.  Experiencia Laboral:  Desarrollador Web Senior, WebTech Solutions Lideré el equipo de desarrollo en la implementación de una plataforma de comercio electrónico utilizando React.js, incrementando las ventas en un 40%. Educación:  Ingeniería Informática, Universidad Autónoma de Madrid (UAM) Aptitudes:  Desarrollo Web Full-Stack Gestión de Proyectos Liderazgo Técnico   

------------
Oferta con id 2
Desarrollador de Software Frontend - Experiencia de Usuario (UX)  Acerca del empleo: Estamos en busca de un Desarrollador de Software Frontend a

**Extraer en funcion de una oferta de trabajo los cv mas similares**

In [18]:
def mean_embedding_job_id(id_job):
    # Calcular la media de los embeddings
    mean_embedding_job = np.mean(embeddings_job[id_job], axis=0)
    return mean_embedding_job

In [22]:
def id_to_textCV(list_cv_id_simil, df_cvs):
    id_cv = [id_[0] for id_ in list_cv_id_simil]
    text_cv = df_cvs.loc[id_cv, 'cv']
    return text_cv

In [23]:
def shows_cvs_for_this_job(id_job, threshold):
    # retornamos el embedding de la oferta de trabajo con id dado
    embedding_job=mean_embedding_job_id(id_job)

    # Iteramos sobre todos los cv de nuestros embeddings, almacenando la similitud dada por la oferta pasada por parametro
    list_cosine_cv = []
    for cv in range(len(embeddings_cv)):
        # Calcular la media de los embeddings
        mean_embedding_cv = np.mean(embeddings_cv[cv], axis=0)
        list_cosine_cv.append(cosine_similarity([embedding_job], [mean_embedding_cv])[0, 0])

    # Almacenamos unicamente los cv cuya similitud sea mayor o igual al umbral dado por parametro, ordenado descendentemente
    values_cv_threshold = sorted([[i,value] for i,value in enumerate(list_cosine_cv) if value >= threshold],
                                key=lambda x: x[1], reverse=True)
    
    # Limpiamos los saltos de lineas '\n' de los dataframes originales (sin procesar previamente)
    df_cvs, df_jobs = clean_df_original(df_cv_original, df_job_original)

    # Extraemos el texto correspondiente al id de los cv con el umbral retornado
    text_cv = id_to_textCV(values_cv_threshold, df_cvs)
    text_job = df_jobs['job'][id_job]

    # Pintamos el resultado
    print(f'Para la oferta con id {id_job}:')
    print(text_job)
    print(f'------------\nSe han encontrado {len(values_cv_threshold)} CV que podrian ser aptos para dicha oferta:')
    for i, value in enumerate(text_cv):
        print(f'CV {i+1}: {value}')

In [36]:
shows_cvs_for_this_job(id_job=18,threshold=0.7)# id de la oferta de trabajo/umbral de similitud (0-1)

Para la oferta con id 18:
Científico de Datos en Retail - Especializado en Deep Learning Acerca del empleo: Únete a nuestro equipo como Científico de Datos especializado en Deep Learning para impulsar la innovación en el sector minorista y mejorar la experiencia del cliente.  Responsabilidades:  Investigar, desarrollar y desplegar modelos de deep learning para mejorar la personalización de recomendaciones de productos. Analizar grandes volúmenes de datos transaccionales para identificar patrones de comportamiento del cliente y tendencias de mercado. Colaborar con equipos de negocio para traducir insights de datos en estrategias comerciales efectivas. Cualificación:  Maestría o Doctorado en Ciencia de Datos, Inteligencia Artificial o campo relacionado. Experiencia demostrada en el desarrollo y despliegue de modelos de deep learning en entornos comerciales. Dominio de herramientas y bibliotecas como TensorFlow, PyTorch y Keras. Experiencia previa en el sector minorista o de comercio elec